In [1]:
# I didn't put any google trends data here since we are only exploring amazon reviews in this notebook.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
    
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline


def GetMergedAmazonDataFrames():
#     files_to_use = [r'data\AirPods Pro.csv', r'data\Elite 75t.csv',r'data\Free.csv',
#                    r'data\Galaxy Buds Plus.csv',r'data\M-200.csv',r'data\NC700.csv',
#                    r'data\Soundcore Liberty 2 Pro.csv',r'data\Soundcore Liberty Air 2.csv',r'data\Soundcore Life Q20.csv',
#                    r'data\WF-1000XM3.csv']
    files_to_use = [r'data\AirPods Pro.csv', r'data\Elite 75t.csv',r'data\Free.csv',
                   r'data\Galaxy Buds Plus.csv',r'data\NC700.csv',
                   r'data\Soundcore Liberty 2 Pro.csv',r'data\Soundcore Liberty Air 2.csv',r'data\Soundcore Life Q20.csv',
                   r'data\WF-1000XM3.csv']
    
    amzn_df = pd.read_csv(files_to_use[0],  parse_dates=False)
    for file in files_to_use[1:]:
        df = pd.read_csv(file,  parse_dates=False)
#         amzn_df = pd.merge(amzn_df, df, how='outer',on='review_date')
        amzn_df = pd.concat([amzn_df,df],sort=False)
        
    amzn_df['review_date'] = pd.to_datetime(amzn_df['review_date'], format='%Y-%m-%d')
    amzn_df = amzn_df[['Manufacturer', 'ProductName', 'review_rating', 'verified_purchase',
       'review_date', 'review_title', 'review_text']]
    return amzn_df
    

import numpy as np

def GetAmazonTextReviewsDf():
    ### This function is used to prepare 
    #   the data frame to word and text analysis.. ####
    amzn_df = GetMergedAmazonDataFrames()
    columns = ['Manufacturer', 'ProductName', 'review_rating', 'verified_purchase',
       'review_date', 'review_title', 'review_text']
    
    amzn_df=amzn_df[columns]
    return amzn_df
#     amzn_df['totalReviews'] =
    
def GetAmazonNumericalReviewsDf():
    amzn_df = GetMergedAmazonDataFrames()
    columns = ['Manufacturer', 'ProductName', 'review_rating', 'verified_purchase',
       'review_date']
    
    amzn_df=amzn_df[columns]
    # split into two data sets for purchased and not purchased.
    amzn_purchased = amzn_df.where(amzn_df['verified_purchase']== 1).dropna()
    amzn_notPurchased = amzn_df.where(amzn_df['verified_purchase']== 0).dropna()
    
    tempdf = None
    for product in amzn_purchased['ProductName'].unique():
        df = amzn_purchased.where(amzn_purchased['ProductName'] == product).dropna()
        
        manufacturer = df['Manufacturer'].unique()[0]
        df= df[['review_rating','review_date']]
        
        dfCount = df
        df = df.set_index('review_date').resample('W-SAT').mean().reset_index()#.agg(['mean','count']).reset_index()
#         df['review_rating'] = df['review_rating'].fillna(0)
        df['review_rating_norm'] = Normalize(df['review_rating'].fillna(df['review_rating'].mean()))
        df['ProductName'] = df['review_rating'].apply(lambda x: product)
        df['Manufacturer'] = df['review_rating'].apply(lambda x: manufacturer)
        df['verified_purchase'] = df['review_rating'].apply(lambda x: '1')
        df['count'] = dfCount[['review_rating','review_date']].set_index('review_date').resample('W-SAT').count().reset_index()['review_rating']
#         print(df.head())
        if tempdf is None:
            tempdf = df
        else:
            tempdf = pd.concat([tempdf, df])
            
    
    for product in amzn_notPurchased['ProductName'].unique():
        df = amzn_notPurchased.where(amzn_notPurchased['ProductName'] == product).dropna()
        
        manufacturer = df['Manufacturer'].unique()[0]
        df= df[['review_rating','review_date']]
        
        dfCount = df
        
        
        df = df.set_index('review_date').resample('W-SAT').mean().reset_index()#.agg(['mean','count']).reset_index()
#         df['review_rating'] = df['review_rating'].fillna(0)
        df['review_rating_norm'] = Normalize(df['review_rating'].fillna(df['review_rating'].mean()))
        df['ProductName'] = df['review_rating'].apply(lambda x: product)
        df['Manufacturer'] = df['review_rating'].apply(lambda x: manufacturer)
        df['verified_purchase'] = df['review_rating'].apply(lambda x: '0')
        df['count'] = dfCount[['review_rating','review_date']].set_index('review_date').resample('W-SAT').count().reset_index()['review_rating']
        
#         print(df.head())
        if tempdf is None:
            tempdf = df
        else:
            tempdf = pd.concat([tempdf, df])
            
    return tempdf
    

In [2]:
# Math functions

In [3]:
def Normalize(array):
    return array / np.linalg.norm(array, ord=1)
    
# Sav-golay filter for averaging.
def savGolay(array, window = 9, order =7):
    from scipy.ndimage import convolve1d
    from scipy.signal import savgol_coeffs, savgol_filter
    from scipy.signal._savitzky_golay import _polyder
    return savgol_filter(array, window, order)
   
# Simple Moving Average used in financial data.
def SMA(pandas_df_daily,columnName, dayWindowSize=20):
    daily_close_df = pandas_df_daily[columnName].copy()
    return daily_close_df.rolling(dayWindowSize).mean()

# Functions for working with words

In [85]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

def TokenizeText(text):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
#     text = "This is my text. It includes commas, question marks? and other stuff. Also U.S..".lower()
    tokens = tokenizer.tokenize(text)
    return tokens

def GetWordsAndTheirTags(text):
    text = CleanText(text)
    tokens = TokenizeText(text)
    return nltk.pos_tag(tokens)

def CleanText(text):
    text=str(text)
    text= text.lower().replace("wouldn\'t","would not")
    return text
def helperGetWordsWithTokenType(text, t):   
# #     print(nltk.help.upenn_tagset('PRP$'))
# #     print(text)
#     text = CleanText(text)
#     tokens = TokenizeText(text)
# #     for duo in nltk.pos_tag(tokens):
# #         if duo[1] =='JJ': # adjectives
# #             print(duo)           
#     return [x[0] for x in nltk.pos_tag(tokens) if x[1]=='JJ']
    tokens = TokenizeText(text)          
    return [x[0] for x in nltk.pos_tag(tokens) if x[1] in t]
    
def GetAdjectivesFromText(text):
    text = CleanText(text)
    return helperGetWordsWithTokenType(text,['JJ'])

def GetNounsFromText(text):
    text = CleanText(text)
    return helperGetWordsWithTokenType(text,['NN','NNS'])

def GetVerbssFromText(text):
    text = CleanText(text)
    # Verbs are tricky... we have to add a 'to' before every word in the text, so we can convert words to verbs
    # this will return any words that can be interpreted as a verb even if not used in the text that way.
    # it's difficult to know by the text alone if a word is a verb or not, so this will force it to a verb.
    
    tokens = TokenizeText(text)       
    words = [ 'to '+ x for x in tokens]
#     print(' '.join(words))
    return helperGetWordsWithTokenType(' '.join(words),['VBZ','VB'])

def GetWordPairs(text):
    text = CleanText(text)
    tokens = TokenizeText(text)
    
    #we need to create a new array with pairs of the words in the tokens array.

    pairs = []
    for i in range(0,len(tokens)-1,1):
        firstWord = tokens[i]
        secondWord = tokens[i+1]
        pairs.append(set([firstWord,secondWord]))
    #     print(set([firstWord,secondWord]))
#     print(pairs)
    return pairs

def GetWordTripplets(text):
    text = CleanText(text)
    tokens = TokenizeText(text)
    
    #we need to create a new array with pairs of the words in the tokens array.

    pairs = []
    for i in range(0,len(tokens)-2,1):
        firstWord = tokens[i]
        secondWord = tokens[i+1]
        thirdWord = tokens[i+2]
        pairs.append(set([firstWord,secondWord,thirdWord]))
    #     print(set([firstWord,secondWord]))
#     print(pairs)
    return pairs

from collections import defaultdict
def count_frequency(word_list):
    """ Function input: list object contains strings of individual words.
        Function output: defaultdict() object containing {word, frequency} pairs. """

    freq_counts = defaultdict(int)
    
    for word in word_list:
        freq_counts[word] += 1

    return freq_counts

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AngryGamers\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\AngryGamers\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [81]:
# GetVerbssFromText("This is my hate text. It includes commas, question marks? dog and other stuff. Also U.S..wouldn't it be great")
# GetAdjectivesFromText("This is my text. It includes commas, question marks? and other stuff. Also U.S..".lower())
# #     print(duo[1])
# # tokens
# GetWordsAndTheirTags("This is my hate text. It includes commas, question marks? dog and other stuff. Also U.S..wouldn't it be great")

# text = CleanText("This is my hate text. It includes commas, question marks? dog and other stuff. Also U.S..wouldn't it be great")
# tokens = TokenizeText(text)  

# pairs = []
# for i in range(0,len(tokens)-1,1):
#     firstWord = tokens[i]
#     secondWord = tokens[i+1]
#     pairs.append(set([firstWord,secondWord]))
# #     print(set([firstWord,secondWord]))
# print(pairs)
# # tokens.rotate(1)
GetWordPairs("This is my hate text. It includes commas, question marks? dog and other stuff. Also U.S..wouldn't it be great")

[{'is', 'this'},
 {'is', 'my'},
 {'hate', 'my'},
 {'hate', 'text'},
 {'it', 'text'},
 {'includes', 'it'},
 {'commas', 'includes'},
 {'commas', 'question'},
 {'marks', 'question'},
 {'dog', 'marks'},
 {'and', 'dog'},
 {'and', 'other'},
 {'other', 'stuff'},
 {'also', 'stuff'},
 {'also', 'u'},
 {'s', 'u'},
 {'s', 'would'},
 {'not', 'would'},
 {'it', 'not'},
 {'be', 'it'},
 {'be', 'great'}]

In [86]:
# Functions to tweak pandas data frames.
def AddAdjectiveNounsVerbsToDF(df, textColumn):
    df = df.copy()
#     amzn_df['text_adjectives'] =amzn_df['review_text'].apply(lambda x: GetAdjectivesFromText(str(x).lower()))
#     amzn_df['text_nouns'] =amzn_df['review_text'].apply(lambda x: GetNounsFromText(str(x).lower()))
#     amzn_df['text_verbs'] =amzn_df['review_text'].apply(lambda x: GetVerbssFromText(str(x).lower()))
    df['text_adjectives'] =df[textColumn].apply(lambda x: GetAdjectivesFromText(str(x).lower()))
    df['text_adjectives'] = df['text_adjectives'].apply(lambda row: set([  x for x in row if(len(x) >1)]) )
    
    df['text_nouns'] =df[textColumn].apply(lambda x: GetNounsFromText(str(x).lower()))
    df['text_nouns'] = df['text_nouns'].apply(lambda row: set([  x for x in row if(len(x) >1)]) )
    
    df['text_verbs'] =df[textColumn].apply(lambda x: GetVerbssFromText(str(x).lower()))
    df['text_verbs'] = df['text_verbs'].apply(lambda row: set([  x for x in row if(len(x) >1)]) )
    
    # create word pairs.
    df['text_pairs'] = df[textColumn].apply(lambda row: GetWordPairs(row))
    df['text_tripplets'] = df[textColumn].apply(lambda row: GetWordTripplets(row))
    return df

def GetDataframeOfWords(pandasdf, textColumn):
#     pandasdf['words'] = CreateListForColumn(pandasdf, textColumn)
    pandasdf['words']=pandasdf[textColumn]
    wordArray = pandasdf[pandasdf['words'].map(lambda d: len(d)) > 0]['words'].values
    words = []
    for wordlist in wordArray:
        words.extend(wordlist)
    wordDictionary = count_frequency(words)
    
    df = pd.DataFrame.from_dict(wordDictionary, orient='index',
                       columns=['count']).reset_index()
    df = df.rename(columns={"index": "word"})
    df = df.sort_values(by=['count'], ascending=False)
    
    totalWords = df['count'].sum()
    df['usage'] = df['count']/totalWords
#     df = df.set_index('word')
    
    return df

In [92]:
amzn_df = GetAmazonTextReviewsDf()
amzn_df = AddAdjectiveNounsVerbsToDF(amzn_df,'review_text')
amzn_df.head()

,Manufacturer,ProductName,review_rating,verified_purchase,review_date,review_title,review_text,text_adjectives,text_nouns,text_verbs,text_pairs,text_tripplets
0,Apple,AirPods Pro,5,0,2019-10-30,Airpods Pro deliver😎,This airpods pro do exactly what apple said th...,"{many, different, new}","{money, alot, buds, design, tax, water, apple,...","{buds, design, pro, correctly, noise, know, ma...","[{airpods, this}, {pro, airpods}, {pro, do}, {...","[{airpods, pro, this}, {pro, do, airpods}, {pr..."
1,Apple,AirPods Pro,4,0,2019-10-30,List of Features/Changes compared to Airpods (v2),"Like iphone 11 Pro, Macbook Pro, Airpad Pro; t...","{small, black, previous, closed, pro, audiophi...","{depreciation, place, phones, lows, range, hea...","{depreciation, place, audiophile, squeeze, ran...","[{iphone, like}, {iphone, 11}, {pro, 11}, {pro...","[{iphone, like, 11}, {pro, iphone, 11}, {pro, ..."
2,Apple,AirPods Pro,3,0,2019-10-30,Cool but expensive,First review yeh heard about these excited but...,{excited},"{review, dollars, airpods}","{review, like, yeh, heard, get, less, first}","[{review, first}, {review, yeh}, {yeh, heard},...","[{review, first, yeh}, {heard, review, yeh}, {..."
3,Apple,AirPods Pro,5,0,2019-10-30,Lit,They fire,{},{},{fire},"[{they, fire}]",[]
4,Apple,AirPods Pro,5,1,2019-10-31,Saved my marriage,"With the new transparency mode, I can listen t...",{new},"{street, boys, music, transparency, car, tim, ...","{street, boys, be, transparency, listen, ran, ...","[{the, with}, {the, new}, {new, transparency},...","[{the, new, with}, {the, new, transparency}, {..."


In [94]:
amzn_df.head()

,Manufacturer,ProductName,review_rating,verified_purchase,review_date,review_title,review_text,text_adjectives,text_nouns,text_verbs,text_pairs,text_tripplets,words
0,Apple,AirPods Pro,5,0,2019-10-30,Airpods Pro deliver😎,This airpods pro do exactly what apple said th...,"{many, different, new}","{money, alot, buds, design, tax, water, apple,...","{buds, design, pro, correctly, noise, know, ma...","[{airpods, this}, {pro, airpods}, {pro, do}, {...","[{airpods, pro, this}, {pro, do, airpods}, {pr...","{buds, design, pro, correctly, noise, know, ma..."
1,Apple,AirPods Pro,4,0,2019-10-30,List of Features/Changes compared to Airpods (v2),"Like iphone 11 Pro, Macbook Pro, Airpad Pro; t...","{small, black, previous, closed, pro, audiophi...","{depreciation, place, phones, lows, range, hea...","{depreciation, place, audiophile, squeeze, ran...","[{iphone, like}, {iphone, 11}, {pro, 11}, {pro...","[{iphone, like, 11}, {pro, iphone, 11}, {pro, ...","{depreciation, place, audiophile, squeeze, ran..."
2,Apple,AirPods Pro,3,0,2019-10-30,Cool but expensive,First review yeh heard about these excited but...,{excited},"{review, dollars, airpods}","{review, like, yeh, heard, get, less, first}","[{review, first}, {review, yeh}, {yeh, heard},...","[{review, first, yeh}, {heard, review, yeh}, {...","{review, like, yeh, heard, get, less, first}"
3,Apple,AirPods Pro,5,0,2019-10-30,Lit,They fire,{},{},{fire},"[{they, fire}]",[],{fire}
4,Apple,AirPods Pro,5,1,2019-10-31,Saved my marriage,"With the new transparency mode, I can listen t...",{new},"{street, boys, music, transparency, car, tim, ...","{street, boys, be, transparency, listen, ran, ...","[{the, with}, {the, new}, {new, transparency},...","[{the, new, with}, {the, new, transparency}, {...","{street, boys, be, transparency, listen, ran, ..."


In [107]:

# https://www.mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas/

# GetDataframeOfWords(amzn_df,'text_adjectives')
# GetDataframeOfWords(amzn_df,'text_verbs')
def count_frequency_dictionary(word_list):
    """ Function input: list object contains strings of individual words.
        Function output: defaultdict() object containing {word, frequency} pairs. """

    freq_counts = [defaultdict(int)]
    
    for word in word_list:
        print(word)
        s1,s2 = word
        freq_counts[s1+'-'+s2] += 1

    return freq_counts

# GetDataframeOfWords(amzn_df[:50],'text_pairs')
# from collections import Counter
# Counter(amzn_df[:50]['text_pairs'])

# amzn_df[:50]['text_pairs'].value_counts()


amzn_df[:50]['text_pairs'].apply(pd.Series)

# count_frequency_dictionary(amzn_df[:50]['text_pairs'].value_counts())
# dictionary = {}
# for elem in amzn_df[:50]['text_pairs']:
#     dictionary
#     print(elem)

,0,1,2,3,4,5,6,7,8,9,...,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282
0,"{airpods, this}","{pro, airpods}","{pro, do}","{do, exactly}","{what, exactly}","{what, apple}","{apple, said}","{they, said}","{they, will}","{do, will}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{iphone, like}","{iphone, 11}","{pro, 11}","{pro, macbook}","{pro, macbook}","{pro, airpad}","{pro, airpad}","{pro, the}","{the, pro}","{pro, denotes}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{review, first}","{review, yeh}","{yeh, heard}","{about, heard}","{these, about}","{these, excited}","{but, excited}","{but, i}","{i, d}","{rather, d}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{they, fire}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{the, with}","{the, new}","{new, transparency}","{transparency, mode}","{i, mode}","{can, i}","{can, listen}","{to, listen}","{to, music}","{music, knowing}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"{they, feel}","{a, feel}","{a, lot}","{snugger, lot}","{snugger, 100}","{worth, 100}","{worth, it}",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"{the, airpods}","{pro, airpods}","{pro, is}","{an, is}","{an, amazing}","{amazing, upgrade}","{from, upgrade}","{from, airpods}","{now, airpods}","{now, with}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"{these, earphones}","{earphones, are}","{top, are}","{top, of}","{the, of}","{the, line}","{line, here}","{here, are}","{some, are}","{some, highlighted}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"{hit, apple}","{hit, a}","{a, home}","{run, home}","{with, run}","{these, with}","{these, i}","{wanted, i}","{wanted, the}","{the, original}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
